In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import inspect

# BatchNorm

## nn.BatchNorm2d

In [12]:
inp_2d = torch.randn(20, 100, 35, 45)*10000

In [61]:
# BatchNorm2d: affine=False, track_running_stats=False
m_bn2d_1 = nn.BatchNorm2d(100, affine=False,track_running_stats=False,eps=0)
out1_bn2d_1 = m_bn2d_1(inp_2d)
out2_bn2d_1 = (inp_2d-inp_2d.mean(dim=(0,2,3),keepdim=True))/(inp_2d.std(dim=(0,2,3),keepdim=True,unbiased=False))
print((out1_bn2d_1-out2_bn2d_1).abs().max())

tensor(4.7684e-07)


In [60]:
# BatchNorm2d: affine=True, track_running_stats=False
m_bn2d_2 = nn.BatchNorm2d(100, affine=True,track_running_stats=False,eps=0)
m_bn2d_2.weight.data.normal_()
m_bn2d_2.bias.data.normal_()
out1_bn2d_2 = m_bn2d_2(inp_2d)
out2_bn2d_2 = (inp_2d-inp_2d.mean(dim=(0,2,3),keepdim=True))/(inp_2d.std(dim=(0,2,3),keepdim=True,unbiased=False))
out2_bn2d_2=(out2_bn2d_2*m_bn2d_2.weight[None,:,None,None])+m_bn2d_2.bias[None,:,None,None]
print((out1_bn2d_2-out2_bn2d_2).abs().max())

False
tensor(1.9073e-06, grad_fn=<MaxBackward1>)


In [78]:
# BatchNorm2d: affine=True, track_running_stats=True
m_bn2d_3 = nn.BatchNorm2d(100, affine=True,track_running_stats=True)
m_bn2d_3.weight.data.normal_()
m_bn2d_3.bias.data.normal_()
m_bn2d_3.train()
out1_bn2d_3 = m_bn2d_3(inp_2d)
out2_bn2d_3 = (inp_2d-inp_2d.mean(dim=(0,2,3),keepdim=True))/  \
np.sqrt(inp_2d.var(dim=(0,2,3),keepdim=True,unbiased=False)+m_bn2d_3.eps)

out2_bn2d_3=(out2_bn2d_3*m_bn2d_3.weight[None,:,None,None])+m_bn2d_3.bias[None,:,None,None]
print((out1_bn2d_3-out2_bn2d_3).abs().max())

m_bn2d_3.eval()
out1_bn2d_3 = m_bn2d_3(inp_2d)
out2_bn2d_3=(inp_2d-m_bn2d_3.running_mean[None,:,None,None])/torch.sqrt(m_bn2d_3.running_var[None,:,None,None]+m_bn2d_3.eps)
out2_bn2d_3=(out2_bn2d_3*m_bn2d_3.weight[None,:,None,None])+m_bn2d_3.bias[None,:,None,None]
print((out1_bn2d_3-out2_bn2d_3).abs().max())

tensor(1.9073e-06, grad_fn=<MaxBackward1>)
tensor(5.7220e-06, grad_fn=<MaxBackward1>)


## nn.BatchNorm1d

In [84]:
inp_1d=torch.randn(20,35,100)*1000

In [86]:
m_bn1d=nn.BatchNorm1d(35)
m_bn1d.weight.data.normal_()
m_bn1d.bias.data.normal_()

tensor([-0.2925,  0.7966, -0.7367, -0.6469,  0.7676,  0.3077, -1.1125, -0.8462,
        -2.6437, -0.1936, -0.0811,  0.7204,  1.6462, -1.4753,  0.0702,  0.3489,
         0.6849, -1.6422,  0.2395, -0.4500, -0.2770,  1.3101,  2.3897, -1.8086,
        -1.2602, -0.9702, -0.7591,  0.5849, -0.5498, -1.4289,  2.0940,  1.8600,
        -0.8148, -2.4297,  1.3035])

In [95]:
m_bn1d.train()
out1_bn1d=m_bn1d(inp_1d)
out2_bn1d=(inp_1d-inp_1d.mean(dim=(0,2),keepdim=True))/torch.sqrt(inp_1d.var(dim=(0,2),keepdim=True,unbiased=False)+m_bn1d.eps)
out2_bn1d=out2_bn1d*m_bn1d.weight[None,:,None]+m_bn1d.bias[None,:,None]
print((out1_bn1d-out2_bn1d).abs().max())

m_bn1d.eval()
out1_bn1d=m_bn1d(inp_1d)
out2_bn1d=(inp_1d-m_bn1d.running_mean[None,:,None])/torch.sqrt(m_bn1d.running_var[None,:,None]+m_bn1d.eps)
out2_bn1d=out2_bn1d*m_bn1d.weight[None,:,None]+m_bn1d.bias[None,:,None]
print((out1_bn1d-out2_bn1d).abs().max())

tensor(9.5367e-07, grad_fn=<MaxBackward1>)
tensor(1.9073e-06, grad_fn=<MaxBackward1>)


# InstanceNorm

## nn.InstanceNorm2d

In [24]:
inp_2d = torch.randn(20, 100, 35, 45)*10000

In [25]:
m_in2d=nn.InstanceNorm2d(100,affine=True,track_running_stats=True)
m_in2d.weight.data.normal_()
m_in2d.bias.data.normal_()

tensor([ 0.0197, -0.2637, -0.1892,  0.0169,  0.4926, -0.1371, -0.8044,  0.6312,
        -0.1256,  1.7203, -1.3780, -1.1240, -0.7845,  0.8835,  0.3241,  0.1730,
         0.7913,  0.7250,  0.8551, -1.0900, -0.5454, -1.2181, -1.5272, -0.6476,
        -0.4272, -0.6251,  2.6531,  1.2946,  1.6765, -1.4517, -1.5321, -0.8618,
        -0.4392, -1.0031,  0.1133, -0.6450, -0.4325,  1.0109,  0.4099,  0.5670,
        -0.8679, -1.0621,  0.1606,  0.9934,  0.0745,  0.2096,  1.2718,  0.5572,
         1.7677,  0.3772,  0.8835,  0.3090, -0.2385, -0.9725, -0.2218,  0.1654,
        -0.2244,  0.0734,  0.6947, -1.0699, -1.3008,  0.0041,  2.8195,  0.1725,
        -0.6181,  0.4594, -0.5172,  1.7754, -1.3250,  0.9818,  0.6857, -0.2442,
        -0.4482,  0.3124,  0.1274, -1.5520, -1.3876, -0.3538, -0.5420, -2.7162,
        -0.2971, -0.9713, -1.7945, -0.7317, -0.4323,  0.3192,  0.7896,  0.6281,
         1.6418,  0.0272, -1.5975,  1.8000, -1.2982,  0.1671,  0.8781, -0.3138,
        -0.7231, -0.7242,  0.1802, -1.15

In [34]:
running_mean_last=m_in2d.running_mean.clone()
out1_in2d=m_in2d(inp_2d)
out2_in2d=(inp_2d-inp_2d.mean(dim=(2,3),keepdim=True))/torch.sqrt(inp_2d.var(dim=(2,3),keepdim=True,unbiased=False)+m_in2d.eps)
out2_in2d=out2_in2d*m_in2d.weight[None,:,None,None]+m_in2d.bias[None,:,None,None]
print((out1_in2d-out2_in2d).abs().max())
# The running mean averages across the batch dimension as in the Pytorch C++ code Normalization.cpp
print((m_in2d.running_mean-((1-m_in2d.momentum)*running_mean_last+m_in2d.momentum*inp_2d.mean(dim=(0,2,3),keepdim=False))).abs().max())

tensor(1.9073e-06, grad_fn=<MaxBackward1>)
tensor(7.6294e-06)


## nn.InstanceNorm1d

In [105]:
inp_1d = torch.randn(20, 100, 35)*10000

In [106]:
m_in1d=nn.InstanceNorm1d(100,affine=True,track_running_stats=True)
m_in1d.weight.data.normal_()
m_in1d.bias.data.normal_()

tensor([ 1.3772,  1.3220, -0.1419, -0.9747,  0.5957, -0.3933, -0.8318, -0.2528,
         0.4049,  0.2977,  1.1555,  0.8780,  0.6283, -0.8140,  1.4576,  0.2989,
        -0.4481,  0.5360, -1.5799,  0.1240,  0.3458, -0.5718, -0.1517,  0.5443,
        -0.4988,  0.3967, -0.3738, -1.1931,  0.7341,  0.2635,  1.8829, -0.2817,
        -0.5977, -1.0369,  0.1412,  0.3312,  0.7811, -0.6466,  0.3297, -0.2805,
         1.1135,  0.5369, -0.6329,  0.9211,  0.4083,  0.5627,  0.4439,  2.1032,
         0.4296, -2.0104,  0.2074, -1.2971, -0.1185,  0.6236, -0.6749,  0.0420,
         0.5138, -0.3828, -0.4248, -2.2613, -0.4070, -1.1821, -1.4438,  0.3682,
         1.0151, -0.0926, -0.6053,  0.4498,  0.8613, -0.7680, -1.5074, -0.7037,
         1.4493,  0.7780, -0.6161,  0.8062,  1.5613, -0.1680,  0.0371,  1.3800,
        -1.3432, -1.0245, -0.1183,  0.0307, -0.9204, -0.9425, -2.2076, -0.5570,
         0.8127, -0.2700, -0.4865, -0.0836, -1.3501, -0.9473,  0.1009,  1.0256,
         0.1608,  0.6199, -0.9670, -0.18

In [108]:
out1_in1d=m_in1d(inp_1d)
out2_in1d=(inp_1d-inp_1d.mean(dim=(2,),keepdim=True))/torch.sqrt(inp_1d.var(dim=(2,),keepdim=True,unbiased=False)+m_in1d.eps)
out2_in1d=out2_in1d*m_in1d.weight[None,:,None]+m_in1d.bias[None,:,None]
print((out1_in1d-out2_in1d).abs().max())

tensor(9.5367e-07, grad_fn=<MaxBackward1>)


# LayerNorm

## LayerNorm applied to 2d data

In [ ]:
inp_2d = torch.randn(20, 100, 35, 45)*10000

In [111]:
m_ln2d=nn.LayerNorm(inp_2d.shape[2:])
m_ln2d.weight.data.normal_()
m_ln2d.bias.data.normal_()
print(m_ln2d.weight.shape)

torch.Size([35, 45])


In [112]:
out1_ln2d=m_ln2d(inp_2d)
# Normalization step is like InstanceNorm2d
out2_ln2d=(inp_2d-inp_2d.mean(dim=(2,3),keepdim=True))/torch.sqrt(inp_2d.var(dim=(2,3),keepdim=True,unbiased=False)+m_ln2d.eps)
out2_ln2d=out2_ln2d*m_ln2d.weight[None,None,:,:]+m_ln2d.bias[None,None,:,:]
print((out1_ln2d-out2_ln2d).abs().max())

tensor(8.5831e-06, grad_fn=<MaxBackward1>)


## LayerNorm to 1d data

In [116]:
inp_1d=torch.randn(20,35,100)*1000

In [117]:
m_ln1d=nn.LayerNorm(inp_1d.shape[2:])
m_ln1d.weight.data.normal_()
m_ln1d.bias.data.normal_()
print(m_ln1d.weight.shape)

torch.Size([100])


In [119]:
out1_ln1d=m_ln1d(inp_1d)
out2_ln1d=(inp_1d-inp_1d.mean(dim=(2,),keepdim=True))/torch.sqrt(inp_1d.var(dim=(2,),keepdim=True,unbiased=False)+m_ln1d.eps)
out2_ln1d=out2_ln1d*m_ln1d.weight[None,None,:]+m_ln1d.bias[None,None,:]
print((out2_ln1d-out1_ln1d).abs().max())

tensor(1.9073e-06, grad_fn=<MaxBackward1>)
